# Experiment Approach 1: "TBD"

TODO-Intro

TODO-TOC

## Imports, common settings and environment overview

In [1]:
# ________________________________
# Imports
# ________________________________

# External imports
import mlflow
from mlflow.tracking import MlflowClient
import mysql.connector as connection
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import great_expectations as ge
# Internal imports
# NA

# ________________________________
# Common settings
# ________________________________

# Set tracking uri (tracking server and registry server are the same / not separated)
# No need to set the registry uri in addition, because it defaults to the tracking URI
mlflow.set_tracking_uri("http://mlflow_tracking_server:5555")
# Set experiment
experiment = mlflow.set_experiment("fraud_detection_approach_1")
# Add experiment description
client = MlflowClient()
client.set_experiment_tag(experiment.experiment_id, "mlflow.note.content", "TBD with application data from 2021")

# ________________________________
# Environment overview
# ________________________________

print("#---> Environment overview <---#")

# Get current installed versions
!conda --version
!python3 --version
print("mlflow", mlflow.__version__)
print("great-expectations", ge.__version__)

# Get the current tracking uri
tracking_uri = mlflow.get_tracking_uri()
print(f"Current tracking uri: {tracking_uri}")

# Get the current model registry uri
model_registry_uri = mlflow.get_registry_uri()
print(f"Current model registry uri: {model_registry_uri}")

# Get the current experiment
print(f"Current experiment: {experiment.name} (ID {experiment.experiment_id})")


# ________________________________
# Contants
# ________________________________
MLFLOW_RUN_ID = ""
PATH_TO_DATA = "output/data"
PATH_TO_REPORTS = "output/reports"
OUTPUT_FILE_RAW_ALL = "raw_data.csv"
OUTPUT_FILE_RAW_TRAIN = "raw_data_train.csv"
OUTPUT_FILE_RAW_TEST = "raw_data_test.csv"
OUTPUT_FILE_RAW_SUM_STATS = "raw_data_summary_statistics.csv"
OUTPUT_FILE_PROCESSED_TRAIN = "processed_data_train.csv"
OUTPUT_FILE_PROCESSED_TEST = "processed_data_test.csv"
RAW_DATA_FILES = [f"{PATH_TO_DATA}/raw_data.csv", 
                        f"{PATH_TO_DATA}/raw_data_train.csv", 
                        f"{PATH_TO_DATA}/raw_data_test.csv"]

#---> Environment overview <---#
conda 4.10.3
Python 3.9.7
mlflow 1.25.1
great-expectations 0.15.3
Current tracking uri: http://mlflow_tracking_server:5555
Current model registry uri: http://mlflow_tracking_server:5555
Current experiment: fraud_detection_approach_1 (ID 2)


# Step 1: Ingest Data

## 1a. Load and split raw data

In [2]:
# Open connection to database
db = connection.connect(host="database", 
                        database="db", 
                        user="root", 
                        passwd="",
                        use_pure=True)
# Read data (2021) from application table to pandas dataframe
query = f"SELECT * FROM applications WHERE application_request_time LIKE '2021%' AND is_fraud_verified = 1 AND is_fraud IS NOT NULL;"
raw_data_applications = pd.read_sql(query,db)
# Close connection
db.close() #close the connection

# Create dir if not yet exists
os.makedirs(PATH_TO_DATA, exist_ok=True)

# Split raw data
raw_data_applications_train, raw_data_applications_test = train_test_split(raw_data_applications, test_size=0.30, random_state=42)

# Save to raw_data.csv
raw_data_applications.to_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_ALL), 
    sep=";", 
    encoding="utf-8",
    index=False)

# Save to raw_data_train.csv
raw_data_applications_train.to_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_TRAIN), 
    sep=";", 
    encoding="utf-8",
    index=False)

# Save to raw_data_test.csv
raw_data_applications_test.to_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_TEST), 
    sep=";", 
    encoding="utf-8",
    index=False)

# Start a new MLflow run
with mlflow.start_run(run_name="applications in 2021") as run:
        # Log raw data files
        mlflow.log_artifact(os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_ALL), artifact_path=PATH_TO_DATA)
        mlflow.log_artifact(os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_TRAIN), artifact_path=PATH_TO_DATA)
        mlflow.log_artifact(os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_TEST), artifact_path=PATH_TO_DATA)

# Save and print mlflow run id
MLFLOW_RUN_ID = run.info.run_id
print(MLFLOW_RUN_ID)

b20ba3a5aeba48c6b7687dcc736f4738


In [3]:
raw_data_applications.head()

,trx,title,first_name,last_name,birthday,address,city,phone,email,marital_status,application_request_time,application_type,device_type,browser,is_fraud,is_fraud_verified,is_fraud_predicted,is_fraud_proba
0,1,Mrs.,Giacomo,Reilly,1953-03-28,"1776 Eget, St.",Warren,1-837-714-5407,nibh.enim.gravida@google.org,Married,2021-01-02 08:01:31,finance,mobile,mobile chrome,1,1,None,None
1,2,Mr.,Jordan,May,1985-12-28,408-3420 Penatibus Rd.,Jackson,(251) 517-6514,et.magna@google.couk,Divorced,2021-01-19 20:41:20,consultancy,desktop,chrome,0,1,None,None
2,3,Mr.,Plato,Dale,1962-03-28,261 Mi Road,Annapolis,(757) 282-3722,ut.eros@yahoo.com,Single,2021-01-15 15:06:08,finance,mobile,mobile safari,0,1,None,None
3,4,Dr.,Sophia,Ayala,1969-09-11,"P.O. Box 640, 9058 Est. St.",Colorado Springs,1-766-657-6313,feugiat@aol.couk,Divorced,2021-01-17 15:54:36,consultancy,desktop,safari,1,1,None,None
4,5,Ms.,Hayden,Hunt,1947-08-13,6952 Quisque Avenue,New Orleans,1-161-230-7316,lorem.eget@yahoo.edu,Common-Law,2021-01-12 10:37:19,consultancy,mobile,mobile chrome,1,1,None,None


In [4]:
raw_data_applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   trx                       6000 non-null   int64 
 1   title                     6000 non-null   object
 2   first_name                6000 non-null   object
 3   last_name                 6000 non-null   object
 4   birthday                  6000 non-null   object
 5   address                   6000 non-null   object
 6   city                      6000 non-null   object
 7   phone                     6000 non-null   object
 8   email                     6000 non-null   object
 9   marital_status            6000 non-null   object
 10  application_request_time  6000 non-null   object
 11  application_type          6000 non-null   object
 12  device_type               6000 non-null   object
 13  browser                   6000 non-null   object
 14  is_fraud                

In [5]:
raw_data_applications.describe(include="all")

,trx,title,first_name,last_name,birthday,address,city,phone,email,marital_status,application_request_time,application_type,device_type,browser,is_fraud,is_fraud_verified,is_fraud_predicted,is_fraud_proba
count,6000.000000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000.000000,6000.0,0,0
unique,NaN,4,1147,995,5292,6000,206,6000,5682,4,5999,2,2,6,NaN,NaN,0,0
top,NaN,Ms.,Stuart,Cox,1986-11-05,"1776 Eget, St.",Columbus,1-837-714-5407,nunc@hotmail.edu,Divorced,2021-01-12 10:37:19,consultancy,mobile,firefox,NaN,NaN,NaN,NaN
freq,NaN,1542,17,14,4,1,70,1,5,1577,2,3066,4021,1038,NaN,NaN,NaN,NaN
mean,3000.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.580667,1.0,NaN,NaN
std,1732.195139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.493491,0.0,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN
25%,1500.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0,NaN,NaN
50%,3000.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,NaN
75%,4500.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,NaN


# Step 2: Validate Data

## 2a. Check schema for new data

In [6]:
# Check schema for new data
# Check whether raw data files exist
for file in RAW_DATA_FILES:
    if not os.path.exists(file):
        raise FileNotFoundError(f"raw data file not found {file}")

# Dict for MLflow parameter logging
mlflow_run_parameters = {}

# Check schema for new data
for file in RAW_DATA_FILES:
    # Read raw data file
    raw_data_applications = ge.read_csv(file,
                        sep=";",
                        encoding="utf-8")
    
    # Check 1: specific columns must exist in all raw data files
    result = raw_data_applications.expect_table_columns_to_match_set(
        column_set=["address", "application_request_time", "application_type",
                    "birthday", "browser", "city", "device_type", "email",
                    "first_name", "is_fraud", "is_fraud_verified", "last_name",
                    "marital_status", "phone", "title"
        ], 
        exact_match=False
    )

    if not result.success:
        raise AssertionError(result)

    # Check 2: is_fraud column value must not be null
    result = raw_data_applications.expect_column_values_to_not_be_null(
        column="is_fraud"
    )

    if not result.success:
        raise AssertionError(result)
    
    # Check 3: is_fraud column value must be 0 or 1
    result = raw_data_applications.expect_column_distinct_values_to_be_in_set(
        column="is_fraud",
        value_set=[0,1]
    )

    if not result.success:
        raise AssertionError(result)
    
    # Populate parameter information to dict
    param_name = False
    if file.endswith("raw_data.csv"):
        param_name = "num_instances_all"
    if file.endswith("raw_data_train.csv"):
        param_name = "num_instances_train"
    if file.endswith("raw_data_test.csv"):
        param_name = "num_instances_test"
    mlflow_run_parameters[param_name] = raw_data_applications.shape[0]

# Attach further information to already created MLflow run
with mlflow.start_run(run_id=MLFLOW_RUN_ID) as run:
    # Log batch of parameters and summary stats csv
    mlflow.log_params(mlflow_run_parameters)

## 2b. Compute summary statistics for new data

In [7]:
# Read raw data file (all data)
df_raw_data_all = pd.read_csv(RAW_DATA_FILES[0],
                    sep=";",
                    encoding="utf-8")

# Create csv file based on pd.df.describe including all data types
df_raw_data_all.describe(include="all").T.to_csv(os.path.join(PATH_TO_REPORTS, OUTPUT_FILE_RAW_SUM_STATS), 
                                    encoding="utf-8")

# Attach further information to already created MLflow run
with mlflow.start_run(run_id=MLFLOW_RUN_ID) as run:
    # Log summary statistics csv
    mlflow.log_artifact(os.path.join(PATH_TO_REPORTS, OUTPUT_FILE_RAW_SUM_STATS), artifact_path=PATH_TO_REPORTS)


# Step 3: Preprocess Data 

Input
- raw_data_train.csv
- raw_data_train.csv

Output
- processed_data_train.csv
- processed_data_train.csv




In [8]:
import pandas as pd
import numpy as np

# Read training set
raw_data_applications_train = pd.read_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_TRAIN),
    sep=";",
    encoding="utf-8"
)

# Read test set
raw_data_applications_test = pd.read_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_RAW_TEST),
    sep=";",
    encoding="utf-8"
)

# Define features in scope
features_in_scope = ["address", "application_request_time", "application_type",
                        "birthday", "browser", "city", "device_type", "email",
                        "first_name", "is_fraud", "last_name",
                        "marital_status", "phone", "title"
            ]

In [30]:
# TODO Feature importance


# Chi-square test relationship between feature categorical (x) and output categorical (y)
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder

y = raw_data_applications_train["is_fraud"].copy()
X = raw_data_applications_train[["application_type", "browser", "marital_status", "title", "device_type"]].copy()

from sklearn.preprocessing import LabelEncoder
X["application_type"] = LabelEncoder().fit_transform(X["application_type"])
X["browser"] = LabelEncoder().fit_transform(X["browser"])
X["marital_status"] = LabelEncoder().fit_transform(X["marital_status"])
X["title"] = LabelEncoder().fit_transform(X["title"])
X["device_type"] = LabelEncoder().fit_transform(X["device_type"])


# create and fit feature selector and apply to the data
selector = SelectKBest(chi2, k="all")
selector.fit_transform(X,y)

pd.options.display.float_format = '{:.15f}'.format
np.set_printoptions(suppress=True)
# print Chi-square-statistics- and p-values per feature
pd.DataFrame(
   {
   "features": X.columns.values,
   "scores": selector.scores_,
   "p-values": selector.pvalues_
   }
   ).sort_values("scores", ascending=False)

,features,scores,p-values
1,browser,26.369441843423033,0.000000281966327
4,device_type,22.843275679491697,0.000001757631800
2,marital_status,2.375115790424982,0.123282456689955
0,application_type,0.137067337948395,0.711214058437799
3,title,0.083516141233009,0.772587810485379


In [5]:
# Handle missing data
# TODO
raw_data_applications_train[features_in_scope].isna().sum()



address                     0
application_request_time    0
application_type            0
birthday                    0
browser                     0
city                        0
device_type                 0
email                       0
first_name                  0
is_fraud                    0
last_name                   0
marital_status              0
phone                       0
title                       0
dtype: int64

In [31]:
# Define actions for features in scope
# nothing
# address --> nothing todo
# phone --> nothing todo
# birthday --> nothing todo
# city --> nothing todo
# email --> nothing todo
# first_name --> nothing todo
# is_fraud --> nothing todo
# last_name --> nothing todo


# Create new features based on given features
# application_request_time --> create new weekday column based on application_request_time
# birthday --> nothing todo


# One hot
# application_type --> one hot encoding
# browser --> one hot encoding
# device_type --> one hot encoding
# marital_status --> one hot encoding
# title --> one hot encoding


# Handle missing values
# TODO

# Feature importance
# TODO

# Split/Create features

# Scale features

# Save pipeline / imputer for later live prediction


# Define columns for one hot encoding
cols_to_one_hot = ["application_type", "browser", "device_type", "marital_status", "title"]

# Create new dataframes only with features in scope for later training
processed_data_application_train = raw_data_applications_train[features_in_scope].copy()
processed_data_application_test = raw_data_applications_test[features_in_scope].copy()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# Define column transformer
column_trans = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), cols_to_one_hot),
    remainder="passthrough"
)

# Execute one hot encoding on train and test
transformed_columns_one_hot_train = column_trans.fit_transform(processed_data_application_train)
transformed_columns_one_hot_test = column_trans.fit_transform(processed_data_application_test)

# Get categories from executed one hot encoder
categories_one_hot = np.hstack(column_trans.named_transformers_.onehotencoder.categories_)

# Remove original columns from train and test
processed_data_application_train.drop(columns=cols_to_one_hot, inplace=True)
processed_data_application_test.drop(columns=cols_to_one_hot, inplace=True)

# Append one hot encoded columns again to train and test
for i, category in enumerate(categories_one_hot):
    processed_data_application_train[f"{category}"] = transformed_columns_one_hot_train.T[i]
    processed_data_application_test[f"{category}"] = transformed_columns_one_hot_test.T[i]

# Save to processsed_data_train.csv
processed_data_application_train.to_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_PROCESSED_TRAIN), 
    sep=";", 
    encoding="utf-8",
    index=False)

# Save processed_data_train.csv
processed_data_application_test.to_csv(
    os.path.join(PATH_TO_DATA, OUTPUT_FILE_PROCESSED_TEST), 
    sep=";", 
    encoding="utf-8",
    index=False)


to_drop = ["address",
        "application_request_time",
        "birthday",
        "city",
        "email",
        "first_name",
        "is_fraud",
        "last_name",
        "phone"
        #consultancy
        # finance;chrome;firefox;mobile chrome;mobile firefox;mobile safari;safari;desktop;mobile;Common-Law;Divorced;Married;Single;Dr.;Mr.;Mrs.;Ms.
]


X = processed_data_application_train.copy()
X.drop(columns=to_drop, inplace=True)
y = processed_data_application_train["is_fraud"].copy()



from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X, y)

result_is_fraud = log_reg.predict(X.iloc[1].to_numpy().reshape(1,-1))
result_proba = log_reg.predict_log_proba(X.iloc[1].to_numpy().reshape(1,-1))

print(result_is_fraud)
print(result_proba)

from sklearn.metrics import accuracy_score

X_test = processed_data_application_test.copy()
X_test.drop(columns=to_drop, inplace=True)
y_test = processed_data_application_test["is_fraud"].copy()

y_pred = log_reg.predict(X_test)
score =accuracy_score(y_test,y_pred)


score


[1]
[[-5.217 -0.005]]


0.5788888888888889

In [30]:
raw_data_applications_train[features_in_scope].isna().sum()

address                     0
application_request_time    0
application_type            0
birthday                    0
browser                     0
city                        0
device_type                 0
email                       0
first_name                  0
is_fraud                    0
last_name                   0
marital_status              0
phone                       0
title                       0
dtype: int64